In [1]:
import optuna

from utils_kdd99 import *
print_version()


python:      3.10.5
sklearn:     1.2.2
tensorflow:  2.13.0-rc0
keras:       2.13.1rc0
numpy:       1.23.5
pandas:      1.5.3


In [2]:
# データの読み込み
X, y = load_data(use_full_dataset=False, standard_scale=True, verbose=0, )
# 分類モデルによっては数値ラベルしか対応していないため、目的変数を分類クラスから数値へ変換
y = y.map(lambda x: attack_label_class[x]).map(lambda x: correspondences[x])
y.value_counts()


0    391458
1     97278
2      4107
3      1126
4        52
Name: true_label, dtype: int64

In [3]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_SEED, stratify=y)

In [4]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

In [5]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
def objective_svm(trial: optuna.Trial):
    C = trial.suggest_float('C', 1e-5, 1e5)
    gamma = trial.suggest_float('gamma', 1e-5, 1e5)
    svm = SVC(C=C, gamma=gamma, class_weight='balanced', )
    # 各交差検証における各クラスの正解率の平均
    print('score calculating')
    scores = cross_val_score(svm, x_train, y_train, cv=k_fold, n_jobs=-1)
    return 1 - scores.mean()

study = optuna.create_study()
study.optimize(objective_svm, n_trials=1, timeout=60*5, n_jobs=-1)


[I 2023-08-01 10:14:09,928] A new study created in memory with name: no-name-03729157-af8b-4a9d-84f1-a1c3c8e514ce


score calculating


In [ ]:
study.best_params


In [ ]:
model_svm = SVC(**study.best_params, class_weight='balanced')
model_svm.fit(x_train)
y_pred = model_svm.predict(x_test)
